In [4]:
import pymysql
import pandas as pd
from exploration.config import sql_inst

In [5]:
random_dump_titles = [
    "osu_random_2020_08",
    "osu_random_2020_09",
    "osu_random_2020_10",
    "osu_random_2020_11",
    "osu_random_2020_12",
    "osu_random_2021_01"
]

In [6]:
def find_repeat_users(dump_titles):
    template = "(SELECT user_id FROM {}.osu_user_stats)".format
    QUERY_UNION_USER_ID = '\nUNION ALL\n'.join(map(template, random_dump_titles))

    with sql_inst.cursor() as cursor:
        cursor.execute(
            f"""
            SELECT user_id
            FROM (
                {QUERY_UNION_USER_ID}
            )as USERS_INCLUDING_DUPE
            GROUP BY user_id
            HAVING COUNT(*) > 1
            ORDER BY COUNT(*) DESC 
            """
        )

        return tuple(row[0] for row in cursor)

REPEAT_USERS_TUPLE = find_repeat_users(random_dump_titles)

In [7]:
def get_repeat_freq_table(repeat_users):
    template = ("""
    SELECT score_id, user_id, beatmap_id, enabled_mods
            FROM {}.osu_scores_high
            WHERE
            user_id IN """ + str(REPEAT_USERS_TUPLE)
    ).format

    QUERY_USERS_UNION_SCORES = '\nUNION\n'.join(map(template, random_dump_titles))

    query = f"""
            SELECT B.beatmap_repeats, COUNT(*) as freq FROM (
                SELECT COUNT(*) as beatmap_repeats FROM (
                    {QUERY_USERS_UNION_SCORES}
                ) as A
                GROUP BY A.user_id, A.beatmap_id, A.enabled_mods
            ) as B
            WHERE B.beatmap_repeats > 1
            GROUP BY B.beatmap_repeats
            """

    return pd.read_sql(query, sql_inst)

get_repeat_freq_table(REPEAT_USERS_TUPLE)

,beatmap_repeats,freq
0,2,778
